# Investigating conversion of Elastic ingest pipelines into Cribl worker mappers

## Install prerequisite packages

In [1]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.3/523.3 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 7.8 MB/s eta 0:00:00


## Create a connection to Elasticsearch

In [19]:
from elasticsearch import Elasticsearch
import os

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = os.env('ELASTIC_PASSWORD')

# Create the client instance
client = Elasticsearch(
    "https://es01:9200",
    basic_auth=("elastic", ELASTIC_PASSWORD),
    ca_certs="/certs/ca/ca.crt"
)

# Successful response!
body = client.info().body
# {'name': 'instance-0000000000', 'cluster_name': ...}

body

AttributeError: module 'os' has no attribute 'env'